# AutoGrad

![](./images/tensor_autograd.png)

当我们训练神经网络算法的时候，我们通常要用梯度的反向传播(back propagation)。在反向传播算法中，我们神经网络的参数会根据它们对于最终的损失函数的梯度来调整。

$$\text{output} = f(\text{input}, W)$$
$$\text{Loss} = L(\text{output}, \text{groudtruth}) =  L(f(\text{input}, W), \text{groudtruth})$$
$$W = W - \eta * W_g$$

为了计算公式中的损失函数$L$对于参数$W$的梯度$W_g$，Pytorch提供了一个很强大的自动求梯度的引擎：`torch.autograd`。它支持对于任何计算图的自动梯度计算。

下面我们演示了，如果利用torch.autograd，来求于一个简单的函数的反向求梯度的过程，其中`w`是参数，最终求取的是loss对w的梯度。

In [1]:
import torch

# z = x*w+b, oss = ce_loss(z, y)
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
loss.backward()
print(f"grad of w is {w.grad}")

grad of w is tensor([[0.0068, 0.0260, 0.1471],
        [0.0068, 0.0260, 0.1471],
        [0.0068, 0.0260, 0.1471],
        [0.0068, 0.0260, 0.1471],
        [0.0068, 0.0260, 0.1471]])


在上面的例子中，因为我们想要求取`w`的梯度，所以在创建`w`的时候，设置了`requires_grad=True`，表明我们后续要求取`w`的梯度。

我们可以在创建的时候指定`requires_grad=True`，也可以在创建后，通过`w.requires_grad_(True)`来修改。

# 雅克比矩阵 Jacobian

上面的例子中，我们最终输出的是一个标量（Scalar），对于根节点是一个 Scalar 的反向传播，中间每个 Tensor 对应的梯度的形状和 Tensor 本身是一样的。但当根节点是一个多维的张量时，这时候计算出来对应 Tensor 的梯度是一个 雅克比矩阵。这里，我们会把多维张量展平成为一个 1 维的向量。最终计算出来的雅克比矩阵是一个二维的。

$$\nabla_x = J \cdot \nabla_y$$

In [3]:
x1 = torch.arange(4).float().requires_grad_(True)


def f(x):
    return x * x


jacobian = torch.autograd.functional.jacobian(f, x1)
jacobian

tensor([[0., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 4., 0.],
        [0., 0., 0., 6.]])

In [4]:
y1 = f(x1)
y_grad = torch.ones(y1.shape).float()
y1.backward(y_grad)
torch.allclose(x1.grad.data, jacobian @ y_grad)

True

# 计算图

上面的计算过程，我们可以用下面的计算图来表示：

![Compute Graph](../images/comp-graph.png)

在这个示例网络计算中,w和b是parameters，是我们需要更新优化的部分。需要注意的是 Pytorch 的计算图是动态的，也就是一边创建一边会实时的计算。

我们沿着计算图正方向计算，就可以计算出loss的值 ，而反向传播就是把梯度沿着计算图反方向计算。

下面可以通过一个pytorch中一个标准的计算图计算函数，来大概看出来pytorch是怎么实现正向传播与反向传播的。

In [5]:
class LinearFunction(torch.autograd.Function):
    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0)

        return grad_input, grad_weight, grad_bias

核心就是正向计算时，通过ctx把反向时，需要记录的Tenosr，都保存起来，反向时，再取出来用。

* 每次调用 `torch.autograd.Function` 的 `apply` 方法，都会生成一个新的 `ctx` 对象。
* 这个 `ctx` 与该次 `Function` 调用绑定，并保存与该次调用相关的上下文信息。
* 每个 `Function` 操作都有独立的 `ctx`，即使是相同的操作在不同的调用中也有独立的上下文。

另外 Pytorch 的计算图是保存在 Tensor 的 `grad_fn`中的，它是一个记录了 `BackwardOp`的结构，同时通过 `next_functions` 记录了backward 的整个调用链。

In [6]:
print(z.grad_fn)
print(z.grad_fn.next_functions)

((<SqueezeBackward4 object at 0x7f09ec3875b0>, 0), (<AccumulateGrad object at 0x7f09ec3874f0>, 0))


# 停止梯度的跟踪

在有些时候，我们可能会对我们参数进行一些别的运算，这部分的运算并不是训练的一部分，不需要记录在整个计算图中，进行梯度跟踪。

这时候我们可以用`torch.no_grad()` 块作用域来达到目的。比如在上面的简单的网络示例中，如果我们只是想对我们训练好的模型(w,b)，做一次testing。

In [7]:
print(z.requires_grad)
with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


除此之外，我们还可以使用`detach()`方法，detatch返回的Tensor和原Tensor数据是相同的，但对返回Tensor的任何计算，都不会统计到计算图中。

In [8]:
z = torch.matmul(x, w.detach()) + b.detach()
print(z.requires_grad)

False


但是如果我们对detach后返回的Tenosr进行了修改，那原Tensor也会被对应修改，这时候就破坏了原有的计算图，就会报错。

```text
RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 5]] is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).
```

In [10]:
print(f"x : {x}")
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
# 在形成计算图后，对原计算图中的一个，被用于grad_fn输入的Tensor: x，进行了修改
x1 = x.detach()
x1[0] = 2
print(f"x : {x}")
try:
    loss.backward()
except RuntimeError as err:
    print(err)

x : tensor([2., 1., 1., 1., 1.])
x : tensor([2., 1., 1., 1., 1.])
one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 5]] is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).


可以看出对于`detach()`的调用，pytorch底层应该是有维护有一个变量版本，对于`Tensor.data`属性，则没有这样的功能，所以现在已经不推荐使用`.data`属性了。

一旦backward被调用，这个计算图就被释放了，除非指定backward的参数`retain_graph`，这就可以让我们使用起来更加灵活。

我们可以在每一次forward和backward后，改变网络的结构（DAG的样子），每次forward时会重新构建新的计算图。

In [18]:
x = torch.tensor(2.0, requires_grad=True)
y = x**2  # y = x^2
z = 3 * y  # z = 3 * x^2

# 查看 z 的 grad_fn，计算图存在
print("z.grad_fn:", z.grad_fn)

# 第一次调用 backward，计算梯度，并释放计算图
z.backward()

# 此时 z 的 grad_fn 应该变为 None，因为计算图已经被释放 ===> 实际打印中 grad_fn 还在
print("z.grad_fn after backward:", z.grad_fn)

# 再次调用 backward，会报错，因为计算图已经不存在
try:
    z.backward()
except RuntimeError as e:
    print(e)

z.grad_fn: <MulBackward0 object at 0x7f0b00536b00>
z.grad_fn after backward: <MulBackward0 object at 0x7f0b00536b00>
Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.


# 关于 Pytorch Autograd 原理的深度解析

PyTorch Autograd Explained - In-depth Tutorial： https://www.youtube.com/watch?v=MswxJw-8PvE

视频中详细介绍了：

1. 计算图的创建过程
2. grad_fn 的机制
3. next_functions 中存的是什么
4. 在形成计算图片，能不能对计算图中的 tensor 进行修改
5. detach 的影响

接下来，让我们通过几个例子来分析一下，Pytorch 底层的计算图的一些原理。

In [2]:
import torch

a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a * b
c.backward()
print(c.grad_fn)
print(c.grad_fn.next_functions)

((<AccumulateGrad object at 0x7f0dd315aad0>, 0), (<AccumulateGrad object at 0x7f0dd315bac0>, 0))


<img src="../images/autograd_mult.png" width=960px>

从上面的图上，我们分析以下几个重点问题。

## 什么是计算图中的叶子节点

在 PyTorch 中，`is_leaf` 属性用于判断一个张量是否为“叶子节点”。叶子节点是指没有父节点的张量，即没有由其他张量通过运算生成的张量。`is_leaf` 为 `True` 的张量通常是直接创建的张量：比如使用 `torch.tensor()`、`torch.randn()` 等函数。

当 `requires_grad=True` 时，PyTorch 会自动追踪该张量的计算图，如果这个张量是叶子节点，那么它的梯度会在反向传播时保存下来。叶子节点往往是计算图的 `Forward` 起始点，在反向传播时，只有这些节点的梯度会被保留。中间节点虽然也会有梯度计算，但这些梯度在反向传播后会被释放以节省内存。

In [4]:
t1 = torch.tensor(1.0)
print(t1.is_leaf)  # True
t2 = torch.tensor(1.0, requires_grad=True)
print(t2.is_leaf)  # True

True
True


In [9]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a * b
d = c + 1
d.backward()
# 计算图中间的非叶子节点的梯度，在 backward 的时候不会保存下来
print(c.is_leaf, c.grad)  # False, None

c = a + b
# 调用 retain_grad 可以保存中间节点的grad
c.retain_grad()
d = c + 1
d.backward()
print(c.is_leaf, c.grad)  # False, tensor(1.)

False None
False tensor(1.)


/tmp/ipykernel_241690/2863821674.py:7: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(c.is_leaf, c.grad) # False, None


## 什么是 AccumulateGrad

我们可以看到在反向传播的计算图上，`MulBackward` 之后都需要经过 `AccumulatedGrad`才能够得到 `a`和`b`的梯度。这主要是因为在计算图的 Reverse Mode下，每个 Tensor 可能会参与到多个路径上的计算，所以反向传播时需要将梯度进行累加计算。所以从 `c` 反传到`a`和`b`的梯度是通过累加到`a`和`b`原本的梯度上去的。

## Tensor 中的 Version

<img src="../images/autograd_tensor_version.png" width=960px>

In [11]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a * b
d = torch.tensor(4.0, requires_grad=True)
e = c * d
c += 1

try:
    e.backward()
except RuntimeError as e:
    print(e)

one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of AddBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).


在上面的代码中，我们使用中间结果 c 计算了 e，但随后，我们对 c 做了一个inplace 的操作。错误信息所示：当我们调用 `e.backward()`时，计算到`MulBackward`时，它会从 ctx 中获取之前已经保存的 c 和 d，但是发现 c 已经被修改了。这时就是报错。

但这里如果最后计算 `e`的运算是加法，则不会有问题，因为`c`的值不会影响反向传播。

In [13]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a * b
d = torch.tensor(4.0, requires_grad=True)
e = c + d
# 可以正常的对 c 进行 inplace 操作，而不会影响反向的计算图
c += 1
e.backward()

## next_function 中的第二个参数

<img src="../images/autograd_unbind.png" width=960px>

In [16]:
a = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
a1, a2, a3 = a.split(1)
b = a1 * a2
c = b * a3
c.backward()
print(c.grad_fn)
print(c.grad_fn.next_functions)
print(c.grad_fn.next_functions[0][0].next_functions)

((<MulBackward0 object at 0x7f0dd05c0820>, 0), (<SplitBackward0 object at 0x7f0dd05c0e20>, 2))
((<SplitBackward0 object at 0x7f0cb8fe83d0>, 0), (<SplitBackward0 object at 0x7f0cb8fe83d0>, 1))


对于 `split` 这样的算子，它对应的反向传播的算子 `SplitBackward` 除了需要一个反向的梯度外，还需要一个额外的参数，代表当前这个梯度来自哪一部分。